### Imports

In [81]:
import os
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Inputs

In [82]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

In [83]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

In [84]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

### Fuzzy Logic

In [85]:
conf_model = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'conf_model')
conf_efsr = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'conf_efsr')
quality = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'quality')

In [86]:
conf_model['baixa'] = fuzz.trimf(conf_model.universe, [0, 0, 0.5])
conf_model['media'] = fuzz.trimf(conf_model.universe, [0, 0.5, 1])
conf_model['alta'] = fuzz.trimf(conf_model.universe, [0.5, 1, 1])

In [87]:
conf_efsr['baixa'] = fuzz.trimf(conf_efsr.universe, [0, 0, 0.5])
conf_efsr['media'] = fuzz.trimf(conf_efsr.universe, [0, 0.5, 1])
conf_efsr['alta'] = fuzz.trimf(conf_efsr.universe, [0.5, 1, 1])

In [88]:
quality['ruim'] = fuzz.trimf(quality.universe, [0, 0, 0.5])
quality['media'] = fuzz.trimf(quality.universe, [0, 0.5, 1])
quality['boa'] = fuzz.trimf(quality.universe, [0.5, 1, 1])

In [89]:
rules = [
    ctrl.Rule(conf_model['alta'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['alta'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['alta'] & conf_efsr['baixa'], quality['ruim']),
    ctrl.Rule(conf_model['media'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['media'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['media'] & conf_efsr['baixa'], quality['ruim']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['baixa'], quality['ruim'])
]

In [90]:
fuzzy_system = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))

In [91]:
confidence_model = np.where(
    inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [92]:
efsr_min, efsr_max = 0.20, 0.98
confidence_efsr = np.clip((radnet_input_efsr_edge - efsr_min) / (efsr_max - efsr_min), 0, 1)

In [93]:
fused = np.zeros(len(confidence_model))

for i in range(len(confidence_model)):
    fuzzy_system.input['conf_model'] = confidence_model[i]
    fuzzy_system.input['conf_efsr'] = confidence_efsr[i]
    fuzzy_system.compute()
    fused[i] = fuzzy_system.output['quality']

### Inference

In [94]:
cls = (fused >= 0.5).astype(int)
conf = np.where(cls == 1, fused, 1.0 - fused)

In [95]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*62}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
--------------------------------------------------------------
-60    1.3   5      0.45   0.92   1        Bom      0.69      
-67    1.7   3      0.26   0.91   1        Bom      0.68      
-74    4.1   19     0.52   0.43   0        Ruim     0.54      
-64    1.3   15     0.50   0.88   1        Bom      0.65      
-60    3.4   70     0.78   0.50   0        Ruim     0.51      
-70    6.7   146    0.85   0.38   0        Ruim     0.57      
-63    4.9   35     0.83   0.41   0        Ruim     0.55      
-59    2.6   7      0.42   0.44   0        Ruim     0.53      
-76    5.6   56     0.69   0.33   0        Ruim     0.61      
-61    1.5   13     0.22   0.95   1        Bom      0.75      
-52    9.7   31     0.75   0.88   1        Bom      0.64      
-83    7.9   119    0.29   0.33   0        Ruim     0.61      
-93    12.5  47     0.71   0.39   0        Ruim     0.56      
-65    2.8   26     0.70   0.93   1        Bom      0.7

### Save

In [96]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
